In [26]:
# Importamos librerías.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
# sklearn.
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.preprocessing import label_binarize
# Hypertuning utilizando grid search.
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# Oversampler.
from imblearn.over_sampling import  RandomOverSampler
datos_url_pc= "C://Users/maxib/Source/repos/proyecto_final_coder/datos/Base_predictive_maintenance.csv"

In [27]:
# Leer .csv y creo un DF.
df = pd.read_csv(datos_url_pc, delimiter=",", index_col= ["UDI", "Product_ID"])
df.head()

,,Type,Air_temperature_[K],Process_temperature_[K],Rotational_speed_[rpm],Torque_[Nm],Tool_wear_[min],Target,Failure_Type
UDI,Product_ID,,,,,,,,
1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


# Definimos Variables 

In [28]:
# Definimos datos base.
data_x= df.drop(["Failure_Type", "Target"], axis=1)
data_y_ft= df["Failure_Type"]
data_y_t= df["Target"]
# Datos para testear.
data_test_x= pd.get_dummies(data_x)
data_test_y_ft_dum= pd.get_dummies(data_y_ft)

## Resampleo variable Failure type

In [29]:
# Oversampleamos variable Failure Type.
# Preparamos variable Target para entrenar los algoritmos.
X= data_x
y= data_y_ft
# Instantiating the random over sampler.
ros = RandomOverSampler(sampling_strategy={"Heat Dissipation Failure":1000,"Random Failures":1000,"Tool Wear Failure":1000,
                                            "Overstrain Failure":1000, "Power Failure":1000})
# Resampling X, y
X_ros_failure, y_ros_failure = ros.fit_resample(X, y)
X_ros_failure.info()
X_ros_failure["Type"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14652 entries, 0 to 14651
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Type                     14652 non-null  object 
 1   Air_temperature_[K]      14652 non-null  float64
 2   Process_temperature_[K]  14652 non-null  float64
 3   Rotational_speed_[rpm]   14652 non-null  int64  
 4   Torque_[Nm]              14652 non-null  float64
 5   Tool_wear_[min]          14652 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 686.9+ KB


array(['M', 'L', 'H'], dtype=object)

In [30]:
# Adquerimos Dummies de Failure Type (Para Train)
data_y_dum_train= pd.get_dummies(y_ros_failure)
data_X= pd.get_dummies(X_ros_failure)
data_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14652 entries, 0 to 14651
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Air_temperature_[K]      14652 non-null  float64
 1   Process_temperature_[K]  14652 non-null  float64
 2   Rotational_speed_[rpm]   14652 non-null  int64  
 3   Torque_[Nm]              14652 non-null  float64
 4   Tool_wear_[min]          14652 non-null  int64  
 5   Type_H                   14652 non-null  uint8  
 6   Type_L                   14652 non-null  uint8  
 7   Type_M                   14652 non-null  uint8  
dtypes: float64(3), int64(2), uint8(3)
memory usage: 615.4 KB


# Randon Forest Hiper Failure Type

In [48]:
rf= RandomForestClassifier() # Instanciamos el modelo
# UNO VS EL RESTO
model_rf= OneVsRestClassifier(rf)

# Definimos los parámetros de búsqueda
params ={ 
    'n_estimators': [55],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [6,7,8],
    'criterion' :['entropy']
}
# metricas= {'f1_score': make_scorer(f1_score),
#            'precision_score': make_scorer(metrics.precision_score) ,
#            'recall_score': make_scorer(recall_score) }
grid_random_forest = GridSearchCV(estimator = rf,
                                  param_grid = params,
                                #  scoring = "f1",
                                #   refit=f1_score,
                                  cv = 5, 
                                  verbose = 1, # Muestra el resultado en pantalla
                                  n_jobs = -1) # corrida en paralelo

In [33]:
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [53]:
grid_random_forest.fit(data_X, data_y_dum_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [6, 7, 8],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [55]},
             verbose=1)

In [54]:
#Obtenemos el mejor modelo!
grid_random_forest.best_estimator_
print("Mejores parametros: "+str(grid_random_forest.best_params_))
print("Mejor Score: "+str(grid_random_forest.best_score_)+'\n')
scores = pd.DataFrame(grid_random_forest.cv_results_)
scores

Mejores parametros: {'criterion': 'entropy', 'max_depth': 8, 'max_features': 'log2', 'n_estimators': 55}
Mejor Score: 0.6433906586413565



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.448033,0.354284,0.144788,0.041027,entropy,6,sqrt,55,"{'criterion': 'entropy', 'max_depth': 6, 'max_...",0.909928,0.311839,0.869966,0.404437,0.112287,0.521691,0.315411,6
1,2.226008,0.574026,0.140525,0.031025,entropy,6,log2,55,"{'criterion': 'entropy', 'max_depth': 6, 'max_...",0.902764,0.301262,0.897952,0.506826,0.125597,0.546880,0.312831,5
2,1.708331,0.316253,0.134487,0.037163,entropy,7,sqrt,55,"{'criterion': 'entropy', 'max_depth': 7, 'max_...",0.926305,0.324804,0.871331,0.464505,0.149829,0.547355,0.304297,4
3,2.015467,0.292028,0.142776,0.037437,entropy,7,log2,55,"{'criterion': 'entropy', 'max_depth': 7, 'max_...",0.908905,0.363357,0.928669,0.607167,0.233447,0.608309,0.280540,2
4,1.795158,0.337726,0.112513,0.008443,entropy,8,sqrt,55,"{'criterion': 'entropy', 'max_depth': 8, 'max_...",0.935176,0.381440,0.868259,0.608191,0.232082,0.605030,0.271064,3
5,1.958076,0.308262,0.099031,0.018186,entropy,8,log2,55,"{'criterion': 'entropy', 'max_depth': 8, 'max_...",0.926987,0.399864,0.891809,0.663481,0.334812,0.643391,0.243749,1


In [55]:
#Predicción de casos nuevos
# Prediccion en Train
y_train_pred= grid_random_forest.predict(data_X)
# Prediccion en Test
y_test_pred= grid_random_forest.predict(data_test_x)
# 
y_score= grid_random_forest.best_estimator_.predict_proba(data_test_x)
#Accuracy
print('Exactitud:', accuracy_score(data_test_y_ft_dum, y_test_pred))

Exactitud: 0.9668


In [56]:
# Analisis score F1
print(f"F1 Score TEST del classificador macro: {f1_score(data_test_y_ft_dum, y_test_pred, average='macro')}")

print(f"F1 Score TEST del classificador micro: {f1_score(data_test_y_ft_dum, y_test_pred, average='micro')}")

print(f"F1 Score TEST del classificador weighted: {f1_score(data_test_y_ft_dum, y_test_pred, average='weighted')}")

print(f"F1 Score TEST del classificador: {f1_score(data_test_y_ft_dum, y_test_pred, average=None)}")
# Analisis jaccard
print(f"Jaccard Score TEST del classificador macro: {metrics.jaccard_score(data_test_y_ft_dum, y_test_pred, average='macro')}")

print(f"Jaccard Score TEST del classificador micro: {metrics.jaccard_score(data_test_y_ft_dum, y_test_pred, average='micro')}")

print(f"Jaccard Score TEST del classificador weighted: {metrics.jaccard_score(data_test_y_ft_dum, y_test_pred, average='weighted')}")


F1 Score TEST del classificador macro: 0.7525137279350389
F1 Score TEST del classificador micro: 0.9705365657782461
F1 Score TEST del classificador weighted: 0.9758326359514438
F1 Score TEST del classificador: [0.72258065 0.98257436 0.92857143 1.         0.5        0.38135593]
Jaccard Score TEST del classificador macro: 0.661167381258212
Jaccard Score TEST del classificador micro: 0.9427596294490492
Jaccard Score TEST del classificador weighted: 0.9563932427690203


In [57]:
metrics.classification_report(data_test_y_ft_dum, y_test_pred)

C:\Users\maxib\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.57      1.00      0.72       112\n           1       1.00      0.97      0.98      9652\n           2       0.87      1.00      0.93        78\n           3       1.00      1.00      1.00        95\n           4       1.00      0.33      0.50        18\n           5       0.24      1.00      0.38        45\n\n   micro avg       0.97      0.97      0.97     10000\n   macro avg       0.78      0.88      0.75     10000\nweighted avg       0.99      0.97      0.98     10000\n samples avg       0.97      0.97      0.97     10000\n'